# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import datetime
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')
#print(song_files)

In [5]:
filepath = song_files[1]


In [6]:
df = pd.read_json(filepath,lines='true')
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title','artist_id','year','duration']]
#print(song_data.values[0])
song_data=song_data.values[0].tolist()
#print(song_data)



#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
df.head()
artist_data=df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
#print(artist_data.values[0])
artist_data=artist_data.values[0].tolist()
#print(artist_data)




#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data/')
#print(len(log_files))

In [12]:
filepath = log_files[0]
#print(filepath)

In [13]:
df = pd.read_json(filepath,lines='true')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [15]:
df['ts']=pd.to_datetime(df['ts'],unit='ms') 
#print(df)



In [16]:
time_data=[]
for i in range(len(df)):
   time_data.append([df.ts.values[i], df.ts.dt.hour.values[i], df.ts.dt.day.values[i], df.ts.dt.dayofweek.values[i], df.ts.dt.month.values[i], df.ts.dt.year.values[i], df.ts.dt.weekday.values[i]])
    #print(df.ts.dt.hour.values[i])

#print(df)
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [17]:
time_df = pd.DataFrame(time_data,columns=['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])
print(time_df)

                 start_time  hour  day  week  month  year  weekday
0   2018-11-30 00:22:07.796     0   30     4     11  2018        4
1   2018-11-30 01:08:41.796     1   30     4     11  2018        4
2   2018-11-30 01:12:48.796     1   30     4     11  2018        4
3   2018-11-30 01:17:05.796     1   30     4     11  2018        4
4   2018-11-30 01:20:56.796     1   30     4     11  2018        4
5   2018-11-30 01:24:32.796     1   30     4     11  2018        4
6   2018-11-30 01:28:19.796     1   30     4     11  2018        4
7   2018-11-30 01:32:00.796     1   30     4     11  2018        4
8   2018-11-30 01:38:17.796     1   30     4     11  2018        4
9   2018-11-30 01:39:53.796     1   30     4     11  2018        4
10  2018-11-30 01:44:10.796     1   30     4     11  2018        4
11  2018-11-30 01:48:36.796     1   30     4     11  2018        4
12  2018-11-30 01:50:28.796     1   30     4     11  2018        4
13  2018-11-30 01:55:35.796     1   30     4     11  2018     

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
df.head()
user_df = df[['userId','firstName','lastName','gender','level']]
print(user_df)

    userId firstName lastName gender level
0       91    Jayden     Bell      M  free
1       73     Jacob    Klein      M  paid
2       73     Jacob    Klein      M  paid
3       73     Jacob    Klein      M  paid
4       73     Jacob    Klein      M  paid
5       73     Jacob    Klein      M  paid
6       73     Jacob    Klein      M  paid
7       73     Jacob    Klein      M  paid
9       73     Jacob    Klein      M  paid
10      73     Jacob    Klein      M  paid
11      73     Jacob    Klein      M  paid
12      73     Jacob    Klein      M  paid
13      73     Jacob    Klein      M  paid
14      73     Jacob    Klein      M  paid
15      73     Jacob    Klein      M  paid
16      73     Jacob    Klein      M  paid
17      73     Jacob    Klein      M  paid
18      73     Jacob    Klein      M  paid
19      73     Jacob    Klein      M  paid
23      86     Aiden     Hess      M  free
24      86     Aiden     Hess      M  free
25      86     Aiden     Hess      M  free
26      86 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
c=0
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
        #print(songid,artistid)

    # insert songplay record
    #songplay_data = df[['ts','userId','level','song','artist','sessionId','location','userAgent']]
    #pd.to_datetime(str(date))
    #str(t)[:10]
    songplay_data = [pd.to_datetime(row.ts, unit='ms'),df.userId.values[c],df.level.values[c],df.song.values[c],df.artist.values[c],df.sessionId.values[c],df.location.values[c],df.userAgent.values[c]]
    print(songplay_data)
    
    #songplay_data1=['abc',91,'free','abc','abc',829,'abc','abc']
    
    c=c+1
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

[Timestamp('2018-11-30 00:22:07.796000'), '91', 'free', "Jim Henson's Dead", 'Stephen Lynch', 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)']


ProgrammingError: can't adapt type 'numpy.int64'

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [23]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.